In [46]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models, optimizers
from PIL import Image

# Définir les chemins des données
train_dir =r'C:\Users\LENOVO\Downloads\archive\chest_xray\train' 
val_dir = r'C:\Users\LENOVO\Downloads\archive\chest_xray\val' 
test_dir = r'C:\Users\LENOVO\Downloads\archive\chest_xray\test'
# Création des générateurs d'images
train_datagen = ImageDataGenerator(rescale=1./255) 
val_datagen = ImageDataGenerator(rescale=1./255) 
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
train_dir,
target_size=(128, 128),
batch_size=32,
class_mode='binary')

test_generator = train_datagen.flow_from_directory(
test_dir,
target_size=(128, 128),
batch_size=32,
class_mode='binary')

val_generator = train_datagen.flow_from_directory(
val_dir,
target_size=(128, 128),
batch_size=32,
class_mode='binary')

import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.layers import Layer
from tensorflow.keras import backend as K

class CapsuleLayer(Layer):
    def __init__(self, num_capsules, dim_capsule, routings, **kwargs):
        super(CapsuleLayer, self).__init__(**kwargs)
        self.num_capsules = num_capsules
        self.dim_capsule = dim_capsule
        self.routings = routings

    def build(self, input_shape):
        if isinstance(input_shape, list):
            input_shape = tuple(input_shape)

        assert len(input_shape) == 3
        self.input_num_capsules = input_shape[1]
        self.input_dim_capsule = input_shape[2]

        # Initialisation des poids pour le routage des capsules
        self.W = self.add_weight(
            shape=[self.num_capsules, self.input_num_capsules, self.dim_capsule, self.input_dim_capsule],
            initializer='glorot_uniform',
            name='W'
        )
        super(CapsuleLayer, self).build(input_shape)

    def call(self, inputs):
        inputs_shape = K.int_shape(inputs)
        print("Inputs shape:", inputs_shape)
        print("Weights shape:", K.int_shape(self.W))

        # Ajustement de la forme de self.W si nécessaire
        self.W = K.reshape(self.W, (self.num_capsules, self.input_num_capsules, self.dim_capsule, self.input_dim_capsule))

        # Expansion des dimensions et répétition
        inputs = K.expand_dims(inputs, 2)  # Shape: (batch_size, num_capsules, 1, input_dim_capsule)
        inputs = K.tile(inputs, [1, 1, self.num_capsules, 1])  # Répéter pour le routage

        # Assurez-vous que les dimensions correspondent avant la multiplication matricielle
        assert K.int_shape(inputs)[-1] == K.int_shape(self.W)[-1], "Dimension mismatch for dot product."

        inputs = K.dot(inputs, self.W)  # Shape: (batch_size, num_capsules, num_capsules, dim_capsule)

        # Implémentation simplifiée de l'algorithme de routage
        inputs = K.sum(inputs, axis=2)
        return inputs

def CapsNet(input_shape, num_classes):
    x = layers.Input(shape=input_shape)
    conv1 = layers.Conv2D(256, (9, 9), strides=1, padding='valid', activation='relu')(x)
    primarycaps = layers.Conv2D(8, (9, 9), strides=2, padding='valid', activation='relu')(conv1)
    primarycaps = layers.Reshape(target_shape=[-1, 8])(primarycaps)
    primarycaps = layers.Lambda(lambda x: K.sqrt(K.sum(K.square(x), axis=-1, keepdims=True)))(primarycaps)
    digitcaps = CapsuleLayer(num_capsules=num_classes, dim_capsule=16, routings=3)(primarycaps)
    out_caps = layers.Lambda(lambda x: K.sqrt(K.sum(K.square(x), axis=-1)))(digitcaps)
    model = models.Model(inputs=x, outputs=out_caps)
    return model

input_shape = (128, 128, 3)
num_classes = 2

model = CapsNet(input_shape=input_shape, num_classes=num_classes)
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='binary_crossentropy',
    metrics=[tf.keras.metrics.Precision()]
)

# Entraînement du modèle
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=val_generator,
    validation_steps=val_generator.samples // val_generator.batch_size,
    epochs=50
)



Found 5216 images belonging to 2 classes.
Found 624 images belonging to 2 classes.
Found 16 images belonging to 2 classes.
Inputs shape: (None, 3136, 1)
Weights shape: (2, 3136, 16, 1)


ValueError: Exception encountered when calling layer "capsule_layer_19" (type CapsuleLayer).

in user code:

    File "C:\Users\LENOVO\AppData\Local\Temp\ipykernel_1516\3551267341.py", line 77, in call  *
        inputs = K.dot(inputs, self.W)  # Shape: (batch_size, num_capsules, num_capsules, dim_capsule)
    File "C:\Users\LENOVO\anaconda3\envs\CI\lib\site-packages\keras\backend.py", line 2450, in dot
        tf.matmul(xt, yt), x_shape[:-1] + y_shape[:-2] + y_shape[-1:]

    ValueError: Dimensions must be equal, but are 1 and 16 for '{{node capsule_layer_19/MatMul}} = MatMul[T=DT_FLOAT, transpose_a=false, transpose_b=false](capsule_layer_19/Reshape_1, capsule_layer_19/Reshape_2)' with input shapes: [?,1], [16,6272].


Call arguments received by layer "capsule_layer_19" (type CapsuleLayer):
  • inputs=tf.Tensor(shape=(None, 3136, 1), dtype=float32)